In [ ]:
# copyright ############################### #
# This file is part of the Xtrack Package.  #
# Copyright (c) CERN, 2021.                 #
# ######################################### #
import xtrack as xt
import xobjects as xo
import xpart as xp

import ducktrack as dtk
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

beta_rel = 0.037
#gamma=  1.00566349259
mass0=938.27208816*1e6 #ev/c^2
c=299792458.0
gamma=1/np.sqrt(1-beta_rel**2)
#beta_rel = np.sqrt(1 - 1/(gamma*gamma))
p0c = mass0*beta_rel*gamma #eV/c
Ek=(gamma-1)*mass0/1e6
E_tot=gamma*mass0
print('gamma',gamma)
print('beta',beta_rel)
print('Ek',Ek)
print('p0c',p0c/1e9)


current = 5*1e-3 #A
cooler_length = 2.33 # m 
radius_e_beam = 25*1e-3  #m

T_perp = 0.01 # <E> [eV] = kb*T
T_l =  0.001 # <E> [eV]
magnetic_field = 0.01  # T for ELENA
magnetic_field_ratio=0
q0=1

c=299792458.0
p0c = mass0*beta_rel*gamma #eV/c

circumference = 30.40531277976 #m
T = circumference/(c*beta_rel)
s_per_turn = T

#from optics repo
# beta_x=1.87350387431
# alfa_x = -0.24136198932
# mu_x=1.21853796862
# beta_y=2.75948574745
# alfa_y = 0.15875010109
# mu_y=0.72001963179

beta_x = 2.1 #m
beta_y = 2.2 #m
disp_x    = 1.5 #m



qx  = 2.36168984503
qy  = 1.38992572490
dqx = -25.93420025117
dqy = -14.90495652098

disp_x    = 0 #m
dqx = 0
dqy = 0

arc = xt.LineSegmentMap(
        qx=qx, qy=qy,
        dqx=dqx, dqy=dqy,
        length=circumference,
        betx=beta_x,
        bety=beta_y
        )



In [2]:
num_particles=10000
# Define the save interval
save_interval = 1000

emittance=10*1e-6 #inital emittance

sigma_x = np.sqrt(beta_x*emittance)
sigma_px = np.sqrt(emittance*1/beta_x)
sigma_y = np.sqrt(beta_y*emittance)
sigma_py = np.sqrt(emittance*1/beta_y)
sigma_p = 5e-3

delta = np.random.normal(loc=0.0, scale=sigma_p, size=num_particles)
x = np.random.normal(loc=0.0, scale=sigma_x, size=num_particles) + disp_x * delta
px = np.random.normal(loc=0.0, scale=sigma_px, size=num_particles)
y = np.random.normal(loc=0.0, scale=sigma_y, size=num_particles)
py = np.random.normal(loc=0.0, scale=sigma_py, size=num_particles)

particles = xp.Particles(
    mass0=mass0,
    p0c=p0c,
    q0=q0,
    x=x,
    px=px,
    y=y,
    py=py,
    delta=delta,
    zeta=0
)

cooler = xt.ElectronCooler(current=current, length=cooler_length, radius_e_beam=radius_e_beam,
                            temp_perp=T_perp, temp_long=T_l,
                            magnetic_field=magnetic_field, magnetic_field_ratio=magnetic_field_ratio,
                            neutralisation_space_charge=0)
num_turns = 10*359675  # 1sec
# num_turns = 1

x_list = []
px_list = []
y_list = []
py_list = []
dp_p_list = []
action_x_list = []
emittance_x_list = []
emittance_y_list = []

# Perform the loop
for i in tqdm(range(num_turns)):
    if i % save_interval == 0:
        x_xs = particles.x.copy()
        px_xs = particles.px.copy()
        y_xs = particles.y.copy()
        py_xs = particles.py.copy()
        delta_xs = particles.delta.copy()

        action_x_temp = (x_xs ** 2 / beta_x + beta_x * px_xs ** 2)

        cov00 = np.cov(x_xs, px_xs)
        det00 = np.sqrt(np.linalg.det(cov00))

        cov01 = np.cov(y_xs, py_xs)
        det01 = np.sqrt(np.linalg.det(cov01))

        x_list.append(x_xs)
        px_list.append(px_xs)
        y_list.append(y_xs)
        py_list.append(py_xs)
        dp_p_list.append(np.std(delta_xs))
        action_x_list.append(action_x_temp)
        emittance_x_list.append(det00)
        emittance_y_list.append(det01)

    arc.track(particles)
    cooler.track(particles)

time = np.arange(0, num_turns, save_interval) * s_per_turn

filepath='results/emittance_xsuite.npz'
np.savez(filepath, time=time, emittance_x=emittance_x_list,emittance_y=emittance_y_list,dp_p=dp_p_list)

100%|██████████| 3596750/3596750 [1:17:52<00:00, 769.83it/s]
